In [ ]:
##############################
### Install/load packages ###
#############################
library(ggplot2)
library(cowplot)

In [ ]:
#####################
### Load the data ###
#####################
load("country.report.RData")

#for the codes without description, we put the original code
country.report$description <- ifelse( is.na( country.report$Icd10Name), 
                                     country.report$icd_code, 
                                     country.report$Icd10Name)

In [ ]:
###########################################
### Format the data for ggplot barplot ###
##########################################
country.report <- country.report[order(country.report$siteid, country.report$FractionPatients, decreasing = TRUE), ]

#select the top 5 of each country
selection_list <- by(country.report, country.report["siteid"], head, n=5)
selection_df <- Reduce(rbind, selection_list)


#create a plot for each country
italy <- ggplot(data=selection_df[ selection_df$siteid == "Italy", ], aes(x=reorder(description,FractionPatients), y=FractionPatients)) +
  geom_bar(aes(fill= siteid), stat="identity", position=position_dodge()) + 
  ylim( 0, 1) +
  facet_grid(. ~ siteid ) +
  theme_bw()+
  theme(legend.position = "none", axis.text.y = element_text(size=5),axis.text.x = element_text(size=7), 
        axis.title = element_text(size = 7),strip.text.y = element_text(angle = 0)
  )+
  labs(title = "", 
       x = "", y = "Fraction of patients")+ coord_flip()+
  scale_fill_manual("legend", values = c("Italy" = "#009E73", "France" = "#0072B2", "Germany" = "#E69F00", "USA" = "#D55E00", "Combined" = "#444444"))

france <- ggplot(data=selection_df[ selection_df$siteid == "France", ], aes(x=reorder(description,FractionPatients), y=FractionPatients)) +
  geom_bar(aes(fill= siteid), stat="identity", position=position_dodge()) + 
  facet_grid(. ~ siteid ) +
  ylim( 0, 1) +
  theme_bw()+
  theme(legend.position = "none", axis.text.y = element_text(size=5),axis.text.x = element_text(size=7), 
        axis.title = element_text(size = 7),strip.text.y = element_text(angle = 0)
  )+
  labs(title = "", 
       x = "", y = "Fraction of patients")+ coord_flip()+
  scale_fill_manual("legend", values = c("Italy" = "#009E73", "France" = "#0072B2", "Germany" = "#E69F00", "USA" = "#D55E00", "Combined" = "#444444"))


germany <- ggplot(data=selection_df[ selection_df$siteid == "Germany", ], aes(x=reorder(description,FractionPatients), y=FractionPatients)) +
  geom_bar(aes(fill= siteid), stat="identity", position=position_dodge()) + 
  facet_grid(. ~ siteid ) +
  ylim( 0, 1) +
  theme_bw()+
  theme(legend.position = "none", axis.text.y = element_text(size=5),axis.text.x = element_text(size=7), 
        axis.title = element_text(size = 7),strip.text.y = element_text(angle = 0)
  )+
  labs(title = "", 
       x = "", y = "Fraction of patients")+coord_flip()+
  scale_fill_manual("legend", values = c("Italy" = "#009E73", "France" = "#0072B2", "Germany" = "#E69F00", "USA" = "#D55E00", "Combined" = "#444444"))

usa <- ggplot(data=selection_df[ selection_df$siteid == "USA", ], aes(x=reorder(description,FractionPatients), y=FractionPatients)) +
  geom_bar(aes(fill= siteid), stat="identity", position=position_dodge()) + 
  facet_grid(. ~ siteid ) +
  ylim( 0, 1) +
  theme_bw()+
  theme(legend.position = "none", axis.text.y = element_text(size=5),axis.text.x = element_text(size=7), 
        axis.title = element_text(size = 7),
        strip.text.y = element_text(angle = 0)
  )+
  labs(title = "", 
       x = "", y = "Fraction of patients")+ coord_flip()+
  scale_fill_manual("legend", values = c("Italy" = "#009E73", "France" = "#0072B2", "Germany" = "#E69F00", "USA" = "#D55E00", "Combined" = "#444444"))

In [ ]:
######################################
### Plot all the graphics together ###
######################################
cowplot::plot_grid( italy, france, germany, usa, align = "v", ncol = 2)